In [ ]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# 1. Define the scope
SCOPES = [
    "https://www.googleapis.com/auth/drive"
]

# 2. Authenticate using your Service Account key file
# Replace with the actual path to your JSON key file
SERVICE_ACCOUNT_FILE = '/Users/deepshah/Downloads/tiffinstash-key.json' 

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# 3. Open the Google Sheet
# You can open by title, url, or key. Opening by key is most robust.
# Example URL: https://docs.google.com/spreadsheets/d/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms/edit
SHEET_ID = '1UMfiU20U3DXL5st0PTRTiq5xQ_cB10_nrDvexmrapsQ' 

try:
    # Open the spreadsheet
    sh = client.open_by_key(SHEET_ID)
    
    # Select the specific worksheet (tab) by index (0 is the first one) or name
    worksheet = sh.get_worksheet(3) 
    # OR: worksheet = sh.worksheet("Sheet1")

    # 4. Get all values and convert to DataFrame
    data = worksheet.get_all_records() # Returns a list of dictionaries
    df = pd.DataFrame(data)

    print("Data extracted successfully:")
    print(df.head())

except Exception as e:
    print(f"Error reading Google Sheet: {e}")


Data extracted successfully:
  ORDER ID    DATE           NAME       PHONE                       EMAIL  \
0    OD001  30 Jan        Sherwin  4373449653       sherwin8599@gmail.com   
1    OD002  30 Jan  Prashantbhai   9057823946  Prashant.jhala89@gmail.com   
2    OD003  30 Jan         Tushar  4376695664   tushar_messi10@icloud.com   
3    OD004  30 Jan          Rina   6478806239        ktfood0113@gmail.com   
4    OD005  30 Jan         Jenish  4372424509       Janishkhira@gmail.com   

  HOUSE UNIT NO        ADDRESS LINE 1          CITY      ZIP          SKU  \
0                 3837 Ridgepoit way   Mississauga   L5N 7T7  KHAOT-TS-VD   
1                  14 Brussels Avenue     Brampton   L6Z 0B6  KHAOT-TS-VD   
2                       77 D’Arcy St.      Toronto   M5T 1J9  KHAOT-TS-VD   
3                2910 Termini Terrace  Mississauga   L5M 5S4  KHAOT-TS-VD   
4                   3582 Nablus Gate   Mississauga   L5B 3J9  KHAOT-TS-VD   

   ... CLABL     LABEL  DRIVER NOTE  SELLER N

In [14]:
df

,ORDER ID,DATE,NAME,PHONE,EMAIL,HOUSE UNIT NO,ADDRESS LINE 1,CITY,ZIP,SKU,...,CLABL,LABEL,DRIVER NOTE,SELLER NOTE,UPSTAIR DELIVERY,DELIVERY TIME,QUANTITY,ONGOING,PREMIUM COVERAGE,PREMIUM VALUE
0,OD001,30 Jan,Sherwin,4373449653,sherwin8599@gmail.com,,3837 Ridgepoit way,Mississauga,L5N 7T7,KHAOT-TS-VD,...,0,SD03(KT),,,No,DINNER,2,ONGOING,NO,$0.00
1,OD002,30 Jan,Prashantbhai,9057823946,Prashant.jhala89@gmail.com,,14 Brussels Avenue,Brampton,L6Z 0B6,KHAOT-TS-VD,...,0,SD07(KT),,,No,DINNER,2,ONGOING,NO,$0.00
2,OD003,30 Jan,Tushar,4376695664,tushar_messi10@icloud.com,,77 D’Arcy St.,Toronto,M5T 1J9,KHAOT-TS-VD,...,0,SD16(KT),,,No,DINNER,1,ONGOING,NO,$0.00
3,OD004,30 Jan,Rina,6478806239,ktfood0113@gmail.com,,2910 Termini Terrace,Mississauga,L5M 5S4,KHAOT-TS-VD,...,0,SD19(KT),,,No,DINNER,1,ONGOING,NO,$0.00
4,OD005,30 Jan,Jenish,4372424509,Janishkhira@gmail.com,,3582 Nablus Gate,Mississauga,L5B 3J9,KHAOT-TS-VD,...,0,SD22(KT),,,No,DINNER,2,ONGOING,NO,$0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,OD301,30 Jan,Kaval Modi,2898852972,mummystiffincanada@gmail.com,,1198 Pinegrove Rd,Oakville,l6l2w0,MUMKT-TS-VD,...,0,SD14(MU),Call once u deliver,,No,DINNER,2,ONGOING,NO,$0.00
301,OD302,30 Jan,Abhay Singh,2896750654,mummystiffincanada@gmail.com,146,146 Dowling ave,TORONTO,m6k3a6,MUMKT-TS-VD,...,0,SD15(MU),buzz code 1607 and keep it at the proper place,,No,DINNER,1,ONGOING,NO,$0.00
302,OD303,30 Jan,Nandini SHAH,2896232382,mummystiffincanada@gmail.com,,24 Mabelle Avenue,etobicoke,M9A4X7,MUMKT-TS-VD,...,0,SD16(MU),Buzz code 2932 keep it inside the mail room,,No,DINNER,1,ONGOING,NO,$0.00
303,OD304,30 Jan,Mahek Mehta,4372681303,mummystiffincanada@gmail.com,,7887 weston road,Vaughan,L4L1A6,MUMKT-TS-VD,...,0,SD19(MU),DELIVER @THE BURGER PRIEST STORE,,No,DINNER,1,ONGOING,NO,$0.00


In [15]:
df.shape

(305, 26)